In [50]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math
import numpy as np
from scipy.interpolate import interp1d


def plot_empty_grain_size_distribution_detail():
    x_max = 1  # log scale
    x_min = -3  # log scale

    fig = go.Figure()

    lines_linear = [
        ("Gravel", 4.75),
        ("Sand", 0.75),
        ("Silt", 0.05),
        ("Clay", 0.002),
    ]

    lines = {}

    for index, line in enumerate(lines_linear):
        key = line[0]
        linear_x = line[1]
        if index == 0:
            next = lines_linear[index + 1][1]
            text_log_x = (math.log10(next) + x_max) / 2

        elif index == len(lines_linear) - 1:
            text_log_x = (math.log10(line[1]) + x_min) / 2

        else:
            current = line[1]
            next = lines_linear[index + 1][1]
            text_log_x = (math.log10(current) + math.log10(next)) / 2

        lines[key] = {"linear_x": linear_x, "text_log_x": text_log_x}

    fig.add_hrect(y0=100, y1=110, line_width=1, fillcolor="white", line_color="black")

    for k, v in lines.items():
        if k != "Gravel":
            fig.add_vline(
                x=v["linear_x"], line_width=3, line_dash="dash", line_color="blue"
            )
        fig.add_annotation(text=k, showarrow=False, x=v["text_log_x"], y=105)

    axes_update = dict(
        linewidth=1,
        color="black",
        mirror=True,
        gridwidth=1,
        minor=dict(showgrid=True, gridcolor="black", gridwidth=1),
    )

    fig.update_xaxes(axes_update, range=[-3, 1], title="Particle Size, mm", type="log")
    fig.update_yaxes(axes_update, range=[0, 110])

    return fig


def plot_grain_size_distribution_detail(df, sample_ids):
    df = df[(df["Sample ID"].isin(sample_ids))]

    fig = plot_empty_grain_size_distribution_detail()
    for sample in sample_ids:
        dfx = df[df["Sample ID"] == sample]
        x = dfx["Particle Size"].sort_values()
        y = dfx["Percentage Passing"].sort_values()
        fig.add_trace(go.Scatter(x=x, y=y, mode="lines", name=sample))
    return fig

In [59]:
def calculate_particle_size_distribution_general(particle_size, percentage_passing):
    """
    Accepts two lists of equal length. Returns dictionary of particle size percentages.

    :param particle_size: list, particle size measurements
    :param percentage_passing: list, percent passing at each particle size

    :return particle_size_analysis: dict, summarizing gravel, sand, silt, and clay percentages in sample
    """

    particle_size_analysis = {
        "Gravel": {
            "Boundary": 4.75,
            "Percentage Passing": None,
            "Percentage Contained": None,
        },
        "Sand": {
            "Boundary": 0.75,
            "Percentage Passing": None,
            "Percentage Contained": None,
        },
        "Silt": {
            "Boundary": 0.05,
            "Percentage Passing": None,
            "Percentage Contained": None,
        },
        "Clay": {
            "Boundary": 0.002,
            "Percentage Passing": None,
            "Percentage Contained": None,
        },
    }

    particle_size_boundaries = [
        v["Boundary"] for k, v in particle_size_analysis.items()
    ]

    print(particle_size_boundaries)

    # use Numpy to performa a linear interpolation at each particle size boundary value
    percentage_passing_at_boundaries = np.interp(
        particle_size_boundaries, particle_size, percentage_passing
    )

    # the list from the interpolation is the percent passing, convert this to percent retained
    # list order: [gravel, sand, silt, clay]

    # Gravel
    particle_size_analysis["Gravel"][
        "Percentage Passing"
    ] = percentage_passing_at_boundaries[0]

    particle_size_analysis["Gravel"]["Percentage Contained"] = (
        100 - percentage_passing_at_boundaries[0]
    )

    # Clay
    particle_size_analysis["Clay"][
        "Percentage Passing"
    ] = percentage_passing_at_boundaries[3]

    particle_size_analysis["Clay"][
        "Percentage Contained"
    ] = percentage_passing_at_boundaries[3]

    # Silt
    particle_size_analysis["Silt"][
        "Percentage Passing"
    ] = percentage_passing_at_boundaries[2]

    particle_size_analysis["Silt"]["Percentage Contained"] = (
        particle_size_analysis["Silt"]["Percentage Passing"]
        - particle_size_analysis["Clay"]["Percentage Contained"]
    )

    # Sand
    particle_size_analysis["Sand"][
        "Percentage Passing"
    ] = percentage_passing_at_boundaries[1]

    particle_size_analysis["Sand"]["Percentage Contained"] = 100 - (
        particle_size_analysis["Clay"]["Percentage Contained"]
        + particle_size_analysis["Silt"]["Percentage Contained"]
        + particle_size_analysis["Gravel"]["Percentage Contained"]
    )

    # round calculated values
    for k, v in particle_size_analysis.items():
        for l, w in v.items():
            if l in ["Percentage Passing", "Percentage Contained"]:
                v[l] = round(w, 1)

    return particle_size_analysis

[4.75, 0.75, 0.05, 0.002]
({'Gravel': {'Boundary': 4.75, 'Percentage Passing': 100.0, 'Percentage Contained': 0.0}, 'Sand': {'Boundary': 0.75, 'Percentage Passing': 100.0, 'Percentage Contained': 0.6}, 'Silt': {'Boundary': 0.05, 'Percentage Passing': 99.4, 'Percentage Contained': 59.5}, 'Clay': {'Boundary': 0.002, 'Percentage Passing': 39.9, 'Percentage Contained': 39.9}}, array([100.        , 100.        ,  99.39467312,  39.90540541]))


,Location ID,Depth Top,Sample Reference,Type,Sample ID,Specimen Reference,Depth Specimen Top,Particle Size,Percentage Passing,Test Type,Remarks,Gravel Pct
38,FE85-0005,21.34,SS18,NaN,FE85-0005-SS18,NaN,NaN,0.00122,32,NaN,NaN,NaN
39,FE85-0005,21.34,SS18,NaN,FE85-0005-SS18,NaN,NaN,0.00270,47,NaN,NaN,NaN
40,FE85-0005,21.34,SS18,NaN,FE85-0005-SS18,NaN,NaN,0.00508,67,NaN,NaN,NaN
41,FE85-0005,21.34,SS18,NaN,FE85-0005-SS18,NaN,NaN,0.00678,81,NaN,NaN,NaN
42,FE85-0005,21.34,SS18,NaN,FE85-0005-SS18,NaN,NaN,0.00920,89,NaN,NaN,NaN
43,FE85-0005,21.34,SS18,NaN,FE85-0005-SS18,NaN,NaN,0.01530,96,NaN,NaN,NaN
44,FE85-0005,21.34,SS18,NaN,FE85-0005-SS18,NaN,NaN,0.02400,98,NaN,NaN,NaN
45,FE85-0005,21.34,SS18,NaN,FE85-0005-SS18,NaN,NaN,0.03370,99,NaN,NaN,NaN
46,FE85-0005,21.34,SS18,NaN,FE85-0005-SS18,NaN,NaN,0.07500,100,NaN,NaN,NaN
47,FE85-0005,21.34,SS18,NaN,FE85-0005-SS18,NaN,NaN,0.10600,100,NaN,NaN,NaN


In [45]:
np.interp([0.75], [0.85, 0.425], [100, 100])

array([100.])